This assignment is inspired by: 

- https://www.kaggle.com/code/carlmcbrideellis/an-introduction-to-xgboost-regression
- https://www.kaggle.com/code/dansbecker/xgboost/notebook

In this assignment we will apply XGBoost Regression techniques to predict house prices, based on the famous Kaggle Dataset https://www.kaggle.com/competitions/house-prices-advanced-regression-techniques

Step 1 is to download the dataset.

In [65]:
#=========================================================================
# load up the libraries
#=========================================================================
import pandas  as pd
import numpy   as np
import xgboost as xgb
from sklearn.model_selection import train_test_split

#=========================================================================
# read in the data
#=========================================================================
train_data = pd.read_csv(r'input/house-prices-advanced-regression-techniques/train.csv',index_col=0)
test_data  = pd.read_csv(r'input/house-prices-advanced-regression-techniques/test.csv',index_col=0)

print("train_data = " + str(len(train_data)))
print("test_data = " + str(len(test_data)))

train_data = 1460
test_data = 1459


In [66]:
non_numeric_columns_train = train_data.select_dtypes(exclude=['number']).columns.tolist()
non_numeric_columns_test = test_data.select_dtypes(exclude=['number']).columns.tolist()

new_train_data = pd.get_dummies(train_data, columns=non_numeric_columns_train)
new_test_data = pd.get_dummies(test_data, columns=non_numeric_columns_test)

train_data = new_train_data.dropna(axis=0)
test_data = new_test_data.dropna(axis=0)

print("train_data = " + str(len(train_data)))
print("test_data = " + str(len(test_data)))

train_data = 1121
test_data = 1146


In [67]:
test_data = test_data.iloc[:1121]

print("train_data = " + str(len(train_data)))
print("test_data = " + str(len(test_data)))

train_data = 1121
test_data = 1121


### <center style="background-color:Gainsboro; width:60%;">Feature selection</center>
The purpose of feature selection, as the name suggests, is to only model the most pertinent and important features, thus reducing the computational overhead, and also to alleviate the [curse of dimensionality](https://en.wikipedia.org/wiki/Curse_of_dimensionality). The following are a number of notebooks covering techniques to achieve said goal, all of which use the House Prices data as an example:

* [Feature selection using the Boruta-SHAP package](https://www.kaggle.com/carlmcbrideellis/feature-selection-using-the-boruta-shap-package)
* [Recursive Feature Elimination (RFE) example](https://www.kaggle.com/carlmcbrideellis/recursive-feature-elimination-rfe-example)
* [House Prices: Permutation Importance example](https://www.kaggle.com/carlmcbrideellis/house-prices-permutation-importance-example)
* [Feature importance using the LASSO](https://www.kaggle.com/carlmcbrideellis/feature-importance-using-the-lasso)

In this assignment, we shall use all of the numerical columns, and ignore the categorical features. To encode the categorical features one can use for example [pandas.get_dummies](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.get_dummies.html). 

Our first task is to do Feature Exploration and Selection. 

In [68]:
## Do your work here

non_numeric_columns_train = train_data.select_dtypes(exclude=['number']).columns.tolist()
non_numeric_columns_test = test_data.select_dtypes(exclude=['number']).columns.tolist()

new_train_data = pd.get_dummies(train_data, columns=non_numeric_columns_train)
new_test_data = pd.get_dummies(test_data, columns=non_numeric_columns_test)

# Split the data into input features (X) and target variable (y)
X = new_train_data
y = new_test_data

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### <center style="background-color:Gainsboro; width:60%;">Feature engineering</center>
As mentioned, one aspect of feature engineering is the creation of new features out of existing features. A simple example would be to create a new feature which is the sum of the number of bathrooms in the house:

In [69]:
for df in (X_train, X_test):
    df["n_bathrooms"] = df["BsmtFullBath"] + (df["BsmtHalfBath"]*0.5) + df["FullBath"] + (df["HalfBath"]*0.5)
    df["area_with_basement"]  = df["GrLivArea"] + df["TotalBsmtSF"]

C:\Users\leoau\AppData\Local\Temp\ipykernel_39168\3920428533.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df["n_bathrooms"] = df["BsmtFullBath"] + (df["BsmtHalfBath"]*0.5) + df["FullBath"] + (df["HalfBath"]*0.5)
C:\Users\leoau\AppData\Local\Temp\ipykernel_39168\3920428533.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df["area_with_basement"]  = df["GrLivArea"] + df["TotalBsmtSF"]
C:\Users\leoau\AppData\Local\Temp\ipykernel_39168\3920428533.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually th

Your next task is to apply some feature engineering to prepare for using the XGBoost Estimator to predict house prices.

In [70]:
#code goes here
df['HouseAgeAtSale'] = df['YrSold'] - df['YearBuilt']

area_columns = ['TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'GrLivArea', 'GarageArea']
df['TotalSquareFootage'] = df[area_columns].sum(axis=1)

C:\Users\leoau\AppData\Local\Temp\ipykernel_39168\741354153.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['HouseAgeAtSale'] = df['YrSold'] - df['YearBuilt']
C:\Users\leoau\AppData\Local\Temp\ipykernel_39168\741354153.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['TotalSquareFootage'] = df[area_columns].sum(axis=1)


For more on this fascinating aspect may I recommend the free on-line book ["*Feature Engineering and Selection: A Practical Approach for Predictive Models*"](http://www.feat.engineering/) by Max Kuhn and Kjell Johnson.
### <center style="background-color:Gainsboro; width:60%;">XGBoost estimator</center>
Note that for this competition we use the RMSLE evaluation metric, rather than the default metric, which for regression is the RMSE. For more on the peculiarities of the RMSLE see the Appendix below.

In [74]:
#=========================================================================
# XGBoost regression: 
# Parameters: 
# n_estimators  "Number of gradient boosted trees. Equivalent to number 
#                of boosting rounds."
# learning_rate "Boosting learning rate (also known as “eta”)"
# max_depth     "Maximum depth of a tree. Increasing this value will make 
#                the model more complex and more likely to overfit." 
#=========================================================================
regressor=xgb.XGBRegressor(eval_metric='rmsle')

#=========================================================================
# exhaustively search for the optimal hyperparameters
#=========================================================================
from sklearn.model_selection import GridSearchCV
# set up our search grid
param_grid = {"max_depth":    [4, 5],
              "n_estimators": [500, 600, 700],
              "learning_rate": [0.0001, 0.001, 0.01, 0.1, 0.2, 0.3]}

Can you use grid search to find the optimal hyper parameters?

In [75]:
## put code here

grid_search = GridSearchCV(regressor, param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(X, y)

# Get the best parameters
best_params = grid_search.best_params_

print("Best Parameters:", best_params)



KeyboardInterrupt: 

Now, can you setup a XGBoost Regressor object using your hyperparameters and fit it?

In [ ]:
# put code here

xgboost_model = xgb.XGBRegressor(**best_params)

# Fit the XGBoost model to your data
xgboost_model.fit(X, y)

Finally, can you run it on your test set?

In [64]:
predictions = xgboost_model.predict(X_test)
predictions

NameError: name 'xgboost_model' is not defined

Can you score your solution offline and see how it does?

In [ ]:
from sklearn.metrics import mean_squared_error

# Calculate Mean Squared Error (MSE)
mse = mean_squared_error(y_test, predictions)

print("Mean Squared Error (MSE) on Test Set:", mse)

In [78]:
# read in the ground truth file
solution   = pd.read_csv(r'input/house-prices-advanced-regression-techniques/train.csv') 
y_true     = solution["SalePrice"]

from sklearn.metrics import mean_squared_log_error
RMSLE = np.sqrt( mean_squared_log_error(y_true, predictions) )
print("The score is %.5f" % RMSLE )

Finally, use the below block to prepare your submission

In [ ]:
output = pd.DataFrame({"Id":test_data.index, "SalePrice":predictions})
output.to_csv('submission.csv', index=False)

### <center style="background-color:Gainsboro; width:60%;">Feature importance</center>
Let us also take a very quick look at the feature importance too:

In [ ]:
from xgboost import plot_importance
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
plt.rcParams.update({'font.size': 16})

fig, ax = plt.subplots(figsize=(12,6))
plot_importance(regressor, max_num_features=8, ax=ax)
plt.show();

Where here the `F score` is a measure "*...based on the number of times a variable is selected for splitting, weighted by the squared improvement to the model as a result of each split, and averaged over all trees*." [1] 

Note that these importances are susceptible to small changes in the training data, and it is much better to make use of ["GPU accelerated SHAP values"](https://www.kaggle.com/carlmcbrideellis/gpu-accelerated-shap-values-jane-street-example), incorporated with version 1.3 of XGBoost.

Can you follow the above guide use SHAP values instead of F Score?

In [1]:
# code here

### <center style="background-color:Gainsboro; width:60%;">Appendix: The RMSLE evaluation metric</center>
From the competition [evaluation page](https://www.kaggle.com/c/house-prices-advanced-regression-techniques/overview/evaluation) we see that the metric we are using is the root mean squared logarithmic error (RMSLE), which is given by

$$ {\mathrm {RMSLE}}\,(y, \hat y) = \sqrt{ \frac{1}{n} \sum_{i=1}^n \left(\log (1 + \hat{y}_i) - \log (1 + y_i)\right)^2} $$

where $\hat{y}_i$ is the predicted value of the target for instance $i$, and $y_i$
is the actual value of the target for instance $i$.

It is important to note that, unlike the RMSE, the RMSLE is asymmetric; penalizing much more the underestimated predictions than the overestimated predictions. For example, say the correct value is $y_i = 1000$, then underestimating by 600 is almost twice as bad as overestimating by 600:

In [ ]:
def RSLE(y_hat,y):
    return np.sqrt((np.log1p(y_hat) - np.log1p(y))**2)

print("The RMSLE score is %.3f" % RSLE( 400,1000) )
print("The RMSLE score is %.3f" % RSLE(1600,1000) )

The asymmetry arises because 

$$ \log (1 + \hat{y}_i) - \log (1 + y_i) =  \log \left( \frac{1 + \hat{y}_i}{1 + y_i} \right) $$

so we are essentially looking at ratios, rather than differences such as is the case of the RMSE. We can see the form that this asymmetry takes in the following plot, again using 1000 as our ground truth value:

In [ ]:
plt.rcParams["figure.figsize"] = (7, 4)
x = np.linspace(5,4000,100)
plt.plot(x, RSLE(x,1000))
plt.xlabel('prediction')
plt.ylabel('RMSLE')
plt.show()